In [34]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


100%|██████████| 9.91M/9.91M [00:00<00:00, 37.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.13MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 8.98MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.18MB/s]


In [6]:
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.net(x)


In [7]:
model = MNISTNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [9]:
def train_epoch():
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()
        loss = criterion(model(x), y)
        loss.backward()
        optimizer.step()


In [10]:
def test_model():
    model.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            pred = model(x).argmax(1)
            correct += (pred == y).sum().item()
    print("Accuracy:", correct / len(test_dataset) * 100)


In [11]:
for _ in range(5):
    train_epoch()
test_model()


Accuracy: 97.48


In [12]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.CIFAR10("./data", train=True, download=True, transform=transform_train)
test_data  = datasets.CIFAR10("./data", train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=256)


100%|██████████| 170M/170M [00:02<00:00, 76.1MB/s]


In [13]:
class CIFARNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(64*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [15]:
model = CIFARNet()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()


In [17]:
model.eval()
misclassified = []

with torch.no_grad():
    for x, y in test_loader:
        preds = model(x).argmax(1)
        for i in range(len(y)):
            if preds[i] != y[i]:
                misclassified.append((x[i], preds[i], y[i]))


In [27]:
for _ in range(5):
    train_epoch()
test_model()


Accuracy: 39.839999999999996


In [28]:

train_data1 = datasets.CIFAR100("./data", train=True, download=True, transform=transform_train)
test_data1= datasets.CIFAR100("./data", train=False, download=True, transform=transform_test)


In [29]:
train_loader = DataLoader(train_data1, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_data1, batch_size=256)

In [30]:
class CIFARNet100(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(64*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 100)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        return self.classifier(x)


In [31]:
model = CIFARNet100()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
criterion = nn.CrossEntropyLoss()


In [32]:
model.eval()
misclassified = []

with torch.no_grad():
    for x, y in test_loader:
        preds = model(x).argmax(1)
        for i in range(len(y)):
            if preds[i] != y[i]:
                misclassified.append((x[i], preds[i], y[i]))


In [33]:
for _ in range(5):
    train_epoch()
test_model()


Accuracy: 39.89
